In [26]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import plotly.express as px
import time

In [12]:
train = pd.read_csv("train.csv")
val = pd.read_csv("val.csv")
x_train, y_train = train.drop(columns = ["interest_rate"]), train["interest_rate"]
x_val, y_val = val.drop(columns = ["interest_rate"]), val["interest_rate"]
x_train.shape, x_val.shape, y_train.shape, y_val.shape


((89884, 48), (11235, 48), (89884,), (11235,))

In [14]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression().fit(x_train, y_train)
pred = linreg.predict(x_val)
mse(pred, y_val)


0.4272442699890523

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse
regr = RandomForestRegressorgressor(max_depth=2, random_state=0)
regr.fit(x_train, y_train)
pred = regr.predict(x_val)
mse(pred, y_val)

0.5094010171106792

In [ ]:
# finding hyperparameters
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor(random_state = 42)

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

# Create the random search grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=10, random_state=42, n_jobs = -1)

rf_random.fit(x_train, y_train)



Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [27]:
optimised_random_forest = rf_random.best_params_

start = time.time()
bestRF = RandomForestRegressor(**optimised_random_forest)
bestRF.fit(x_train, y_train)
pred = bestRF.predict(x_val)
mse(pred, y_val)
time.time() - start


21.40061092376709